In [ ]:
#install packages
!python -m pip install -U "pulp==2.6"
!apt-get install -y -qq glpk-utils

!glpsol --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 24.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121730 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpackin

In [ ]:
pip install dash dash-cytoscape networkx matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 39.2 MB/s eta 0:00:00


In [ ]:
# import libraries
import numpy as np
from pulp import LpProblem, LpStatus, lpSum, LpVariable, const, LpConstraint, LpMinimize
from pulp import GLPK
import networkx as nx
import matplotlib.pyplot as plt
from dash import Dash, html, dcc
import dash_cytoscape as cyto
from dash.dependencies import Input, Output

In [ ]:
##### Cloud Network Graph (CNG)

# set of communication links
comm_links = {(1,2),(1,3),(1,4),(2,1),(3,1),(4,1),(2,3),(3,6),(3,2),(6,3),(2,5),(2,6),(6,7),(5,2),(6,2),(7,6)}

# set of computation links
# comp_in links represent the computation resources (memory) used to store the input
comp_in_links = {(2,21),(2,22),(2,23),(3,31),(3,32),(3,33),(3,34),(3,35),(3,36),(6,61),(6,62),(6,63),(1,11),(1,12),(1,13)}
# comp_out links represent the computation resources (CPU) used to create the output
comp_out_links = {(21,2),(22,2),(23,2),(31,3),(32,3),(33,3),(34,3),(35,3),(36,3),(61,6),(62,6),(63,6),(11,1),(12,1),(13,1)}

# set of source and destination links
source_links = {(40,4),(50,5),(70,7),(30,3)}
dest_links = {(4,49),(5,59),(7,79)}

# total set of links in CNG
links = comm_links | comp_in_links | comp_out_links | source_links | dest_links

# set of communication nodes
comm_nodes = {1,2,3,4,5,6,7}

# set of computation nodes
comp_nodes = {11,12,13,21,22,23,61,62,63,31,32,33,34,35,36}

# set of source and dest nodes
source_nodes = {40,50,70,30}
dest_nodes = {49,59,79}

# total set of nodes in CNG
nodes = comm_nodes | comp_nodes | source_nodes | dest_nodes

# set of incoming/outgoing links for each node in the CNG
out_links = {} # key: node u, value: set of outgoing links from node u
in_links = {} # key: node u, value: set of incoming links to node u
for u in nodes:
  outgoing = set() # set of ougoing links from node u
  incoming = set() # set of incoming links to node u
  for e in links:
    if e[0] == u:
      outgoing.add(e)
    if e[1] == u:
      incoming.add(e)
  out_links[u] = outgoing
  in_links[u] = incoming


# Network parameters

# capacity per link
c_res = {} # rate per res allocation block (in Mbps, Mbits, or MHz, for comm, storage, and comp resources, respectively)
c = {} # total capacity (in number of res allocation blocks)
for e in comm_links:
  c_res[e] = 50 # 50 Mbps per comm resource block
  c[e] = 20   # 20 res alloc blocks (so, total capacity in bps is 1 Gbps)
for e in comp_in_links:
  c_res[e] = 100 # 100 Mbits per storage resource block
  c[e] = 100       # 100 storage resource blocks (total capacity 10 Gbits)
for e in comp_out_links:
  c_res[e] = 100  # 100 Mhz per computation resource block
  c[e] = 100  # 100 computation res blocks (total capacity 10 GHz)
for e in source_links | dest_links:
  c_res[e] = 10000 # large enough capacity for source and dest links
  c[e] = 10000

# Max storage and computation capacity per node (sum across all comp links in that node) in res alloc blocks
c_comp_in = {1:1000, 2:1000, 3:1000,6:1000}
c_comp_out = {1:1000, 2:1000, 3:1000, 6:1000}


# cost per link
w = {} # cost per resource allocation block
for e in comm_links:
  w[e] = 100    # 100 cost units per comm resource block (e.g., 100$/10Mbps)
for e in comp_in_links:
  w[e] = 1 # 1 cost unit per storage resource block (e.g., 1$/100Mbit)
for e in comp_out_links:
  w[e] = 10 # 10 cost units per computation resource block (e.g., 10$/100MHz)
for e in source_links | dest_links:
  w[e] = 0 # 0 cost for ingress and egress links

# print cost values
#for j in sorted(w):
#  print(j,w[j])


# propagation delay per communication link
delay_e = {}
for e in links:
  if e in comm_links:
    delay_e[e] = 10
  else:
    delay_e[e] = 0 # set prop delay to zero for source, dest, and computation links


# Create networkx class, which has built-in methods for visualization and graph algorithms (e.g., DFS, BFS, shortest path)
net = nx.DiGraph(comm_links) # this is the network graph (only comm links)
cloudnet = nx.DiGraph(links) # this is the cloud-augmented graph (comm and comp links)

# Set type of link as an attribute of the edges of the cloudnet graph
for e in comm_links:
  cloudnet.edges[e]['type']='comm'
for e in comp_in_links:
  cloudnet.edges[e]['type']='comp_in'
for e in comp_out_links:
  cloudnet.edges[e]['type']='comp_out'
for e in source_links:
  cloudnet.edges[e]['type']='source'
for e in dest_links:
  cloudnet.edges[e]['type']='dest'

print(cloudnet.edges.data())

# Example of how to create subsets of nodes or links matching an attribute
#a = {j for j in cloudnet.edges if cloudnet.edges[j]['type']=='source'}
#print(a)

#plt.subplot(211)
##nx.draw(net, node_color='g', edge_color='b')
#plt.subplot(212)
#nx.draw(cloudnet, node_color='g', edge_color='b')

[(4, 49, {'type': 'dest'}), (4, 1, {'type': 'comm'}), (22, 2, {'type': 'comp_out'}), (2, 5, {'type': 'comm'}), (2, 23, {'type': 'comp_in'}), (2, 1, {'type': 'comm'}), (2, 22, {'type': 'comp_in'}), (2, 3, {'type': 'comm'}), (2, 6, {'type': 'comm'}), (2, 21, {'type': 'comp_in'}), (12, 1, {'type': 'comp_out'}), (1, 3, {'type': 'comm'}), (1, 12, {'type': 'comp_in'}), (1, 2, {'type': 'comm'}), (1, 11, {'type': 'comp_in'}), (1, 4, {'type': 'comm'}), (1, 13, {'type': 'comp_in'}), (3, 1, {'type': 'comm'}), (3, 31, {'type': 'comp_in'}), (3, 34, {'type': 'comp_in'}), (3, 6, {'type': 'comm'}), (3, 33, {'type': 'comp_in'}), (3, 36, {'type': 'comp_in'}), (3, 2, {'type': 'comm'}), (3, 35, {'type': 'comp_in'}), (3, 32, {'type': 'comp_in'}), (5, 59, {'type': 'dest'}), (5, 2, {'type': 'comm'}), (61, 6, {'type': 'comp_out'}), (6, 63, {'type': 'comp_in'}), (6, 2, {'type': 'comm'}), (6, 62, {'type': 'comp_in'}), (6, 7, {'type': 'comm'}), (6, 61, {'type': 'comp_in'}), (6, 3, {'type': 'comm'}), (30, 3, {'ty

In [ ]:
##### Service Graph (SG)


# caching example ( 1 static source,3 caching funcs, 3 proc funcs, 3 dest)
cmds = {(1,2),(1,3),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(7,10)}
source_cmds = {(1,2),(1,3),(1,4)}
static_sources = {(1,2),(1,3),(1,4)}
proc_cmds = {(2,5),(3,6),(4,7)}
dest_cmds = {(5,8),(6,9),(7,10)}
cmds = static_sources | proc_cmds | dest_cmds
source_funcs = {1}
proc_funcs = {2,3,4,5,6,7}
dest_funcs = {8,9,10}
functions = source_cmds | dest_funcs | proc_funcs
objects = {1,2,3,4,5,6,7}
services = {1}
cmds_of_service = {1:cmds}

# set of incoming/outgoing edges for each node in SG
out_cmds = {} # key: function i, value: set of outgoing cmds from function i
in_cmds = {} # key: function i, value: set of incoming cmds to function i
for i in functions:
  outgoing = set() # set of ougoing cmds from function i
  incoming = set() # set of incoming cmds to function i
  for k in cmds:
    if k[0] == i:
      outgoing.add(k)
    if k[1] == i:
      incoming.add(k)
  out_cmds[i] = outgoing
  in_cmds[i] = incoming


# Service parameters

# production, communication, and consumption rares per commodity

R_prod = {}
R_comm = {}
R_cons = {}
for k in static_sources:
  R_prod[k], R_comm[k], R_cons[k] = 15, 15, 15
#for k in cac_cmds:
  #R_prod[k], R_comm[k], R_cons[k] = 0.5*sc_f ,0.006*sc_f, 60
for k in proc_cmds:
  R_prod[k], R_comm[k], R_cons[k] = 50, 50, 50
for k in dest_cmds:
  R_prod[k], R_comm[k], R_cons[k] = 50, 50, 50


# Scaling factors (just to know how functions increase or decrease the input rate, not used in formulation)
comp_scaling = {} # computation scaling factor
comm_scaling = {} # communication scaling factor
for i in proc_funcs:
  for kin in in_cmds[i]:
    for kout in out_cmds[i]:
      comp_scaling[i] = R_cons[kin]/R_comm[kin]
      comm_scaling[i] = R_comm[kout]/R_comm[kin]



# maximum end-to-end delay per destination commodity
d_max = {}
for k in dest_cmds:
  d_max[k] = 200


# Create networkx class for service graph
serv = nx.DiGraph(cmds)

# Set type of commodity as an attribute of edges in service graph
for k in source_cmds:
  serv.edges[k]['type'] ='source'
#for k in cac_cmds:
  #serv.edges[k]['type'] ='caching'
for k in proc_cmds:
  serv.edges[k]['type']='proc'
for k in dest_cmds:
  serv.edges[k]['type']='dest'

# Set object associated with each commodity as an attribute of edges in serv graph
# Note how commodities (2,3) and (2,4), which are both outputs of the static source, are associated to the same information object
serv.edges[(1,2)]['obj'] = 1
serv.edges[(1,3)]['obj'] = 1
serv.edges[(1,4)]['obj'] = 1
serv.edges[(2,5)]['obj'] = 2
serv.edges[(3,6)]['obj'] = 3
serv.edges[(4,7)]['obj'] = 4
serv.edges[(5,8)]['obj'] = 5
serv.edges[(6,9)]['obj'] = 6
serv.edges[(7,10)]['obj'] = 7

print(serv.edges.data())
#nx.draw(serv, node_color='g', edge_color='b')


[(1, 2, {'type': 'source', 'obj': 1}), (1, 4, {'type': 'source', 'obj': 1}), (1, 3, {'type': 'source', 'obj': 1}), (2, 5, {'type': 'proc', 'obj': 2}), (6, 9, {'type': 'dest', 'obj': 6}), (5, 8, {'type': 'dest', 'obj': 5}), (7, 10, {'type': 'dest', 'obj': 7}), (4, 7, {'type': 'proc', 'obj': 4}), (3, 6, {'type': 'proc', 'obj': 3})]


In [ ]:
# Extracting nodes and edges from the service graph for Cytoscape
nodes_cytoscape_serv = [{'data': {'id': str(node), 'label': str(node)}} for node in serv.nodes]
edges_cytoscape_serv = [{'data': {'source': str(edge[0]), 'target': str(edge[1]), 'type': serv.edges[edge]['type']}} for edge in serv.edges]

# Define stylesheet for the graph
g1_stylesheet=[
    # Group selectors Node
    {
        'selector': 'node',
        'style': {
            'background-color': 'red',
            'content': 'data(label)',
            'color' : 'white',
            'shape': 'ellipse',  # Use ellipse or other shapes
            'width': '40px',  # Adjust node width
            'height': '40px',
        }
    },

    # Group selectors Edges
    {
        'selector': 'edge',
        'style': {
            'curve-style': 'bezier',
            'target-arrow-shape': 'triangle',
            'line-color': 'blue',
            'target-arrow-color': 'black'
        }
    },
]

# Create Dash App for Service Graph
app_serv = Dash("ServiceGraph")

# Define the app layout for Service Graph
app_serv.layout = html.Div([
    # Cytoscape Graph
    cyto.Cytoscape(
        id='cytoscape-service-graph',
        layout={'name': 'breadthfirst'},
        style={'width': '100%', 'height': '300px'},
        elements=nodes_cytoscape_serv + edges_cytoscape_serv,
        stylesheet=g1_stylesheet
    ),

    # Dropdown for layout selection
    html.Label('service Graph Layout'),
    dcc.Dropdown(
        id='dropdown-update-layout-serv',
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric', 'breadthfirst']
        ],
        value='breadthfirst',  # Default Value
        style={'display': 'inline-block', 'width': '50%'},
    ),
])

# Callback to update layout based on dropdown value
@app_serv.callback(Output('cytoscape-service-graph', 'layout'),
              [Input('dropdown-update-layout-serv', 'value')])
def update_layout_serv(layout_name):
    return {'name': layout_name}

# Extracting nodes and edges from cloudnet for Cytoscape
nodes_cytoscape_cloudnet = [{'data': {'id': str(node), 'label': str(node)}} for node in nodes]
edges_cytoscape_cloudnet = [{'data': {'source': str(edge[0]), 'target': str(edge[1]), 'type': cloudnet.edges[edge]['type']}} for edge in cloudnet.edges]

# Create Dash App for Cloud Network
app_cloudnet = Dash("CloudNetwork")

# Define the app layout for Cloud Network
app_cloudnet.layout = html.Div([
    # Cytoscape Graph
    cyto.Cytoscape(
        id='cytoscape-first-app',
        layout={'name': 'breadthfirst'},
        style={'width': '100%', 'height': '400px'},
        elements=nodes_cytoscape_cloudnet + edges_cytoscape_cloudnet,
        stylesheet=g1_stylesheet
    ),

    # Dropdown for layout selection
    html.Label('Cloud Network Graph Layout'),
    dcc.Dropdown(
        id='dropdown-update-layout-cloudnet',
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric', 'breadthfirst']
        ],
        value='cose',  # Default Value
        style={'display': 'inline-block', 'width': '50%'},
    ),
])

# Callback to update layout based on dropdown value
@app_cloudnet.callback(Output('cytoscape-first-app', 'layout'),
              [Input('dropdown-update-layout-cloudnet', 'value')])
def update_layout_cloudnet(layout_name):
    return {'name': layout_name}

if __name__ == '__main__':
    app_serv.run_server(debug=True, port=8050)
    app_cloudnet.run_server(debug=True, port=8051)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# source commodities available at each source node
sourceCmds_at_node = {30:{}, 40:{(1,2),(1,3),(1,4)}, 50:{},70:{}}
# destination cmds requested at each destination node
destCmds_at_node = {49:{(5,8)}, 59:{(6,9)}, 79:{(7,10)}}
#procCmds_at_node = {11:{},12:{(2,5)},13:{(4,7)},21:{},22:{(3,6)},23:{},31:{(1,2)},32:{(1,3)},33:{(1,4)},34:{},35:{},36:{}}
procCmds_at_node = {11:{(1,2),(1,3),(1,4)},12:{(2,5),(3,6),(4,7)},13:{},21:{(1,2),(1,3),(1,4)},22:{(2,5),(3,6),(4,7)},23:{},61:{(1,2),(1,3),(1,4)},62:{(2,5),(3,6),(4,7)},63:{},31:{(1,2),(1,3),(1,4)},32:{(2,5),(3,6),(4,7)},33:{},34:{},35:{},36:{}}

In [ ]:
##### Create optimization model

model = LpProblem(name="opt-problem", sense=LpMinimize)


##### Define Decision Variables as dictionaries

# Logical commodity flow variables (binary)
f_k = LpVariable.dicts("f_k", [(e,k) for e in links for k in cmds], lowBound=0, cat='Binary')
# Total object flow variables (real)
f_o = LpVariable.dicts("f_o", [(e,o) for e in links for o in objects], lowBound=0)
# Total link flow variables (real)
f_e = LpVariable.dicts("f_e", [e for e in links], lowBound=0)
# Resource allocation variables (integer)
y = LpVariable.dicts("y", [e for e in links], lowBound=0, cat='Integer')


##### Define Constraints

# Flow Conservation Constraints
for u in comm_nodes:
  for k in cmds:
    model += lpSum([f_k[e,k] for e in in_links[u]]) == lpSum([f_k[e,k] for e in out_links[u]])

# Flow Chaining Constraints
for ein in comp_in_links: # go over all (u,p) links
  for eout in out_links[ein[1]]: # obtain link (p,u) (only one out_link from p)
    for kin in cmds: # go over all cmds
      if kin in procCmds_at_node[eout[0]]: # if cmd can be processed (as input) by p (note that ein[1]=eout[0])
        for kout in out_cmds[kin[1]]: # go over all output cmds
          model += f_k[eout,kout] == f_k[ein,kin] # set chaining constraint
      else: # otherwise, if cmd cannot be processed (as input) by p, set flow variables to zero
        model += f_k[ein,kin] == 0
        for kout in out_cmds[kin[1]]: # including output flow variables
          model += f_k[eout,kout] == 0

# Make sure source cmds cannot be produced at comp_out links
# (not needed for dest_cmds at comp_in_links bc chaining constraints above already constrain flows on comp_in_links to be only of cmds that can be processed)
for e in comp_out_links:
  for k in source_cmds:
    model += f_k[e,k] == 0

# Source Constraints
for e in source_links: # go over links (s,u)
  for k in cmds: # we go over all cmds bc we need to make sure non-source cmds dont get produced at source links
    if k in sourceCmds_at_node[e[0]]: # if k is sourced at node s=e[0], set flow var to 1, ow to 0
      model += f_k[e,k] == 1 #f_su_ij=1 if k sourced at s
    else:
      model += f_k[e,k] == 0 #f_su_ij=0 if k not sourced at s (avoid producing src cmds at other src nodes) or k is not a source cmd

# Destination Constraints
for e in dest_links: # go over links (u,d)
  for k in cmds: # we go over all cmds to make sure that non-dest cmds dont get consumed at dest links
    if k in destCmds_at_node[e[1]]: # if k is requested by d=e[1], set flow var to 1, ow to 0
      model += f_k[e,k] == 1 #f_ud_ij=1 if j is requested by d
    else:
      model += f_k[e,k] == 0 #f_ud_ij=0 for all other cmds not requested by d

# It is important to set source and dest flows to zero at non-source and non-dest nodes to avoid creating fictional dests for the real sources
# and fictional sources for the real dests



# Actual Flow Constraints
for k in cmds:
  o = serv.edges[k]['obj'] # obtain object associated with commodity k
  for e in source_links | comp_out_links:
    model += R_prod[k]*f_k[e,k] <= f_o[e,o]
  for e in comm_links:
    model += R_comm[k]*f_k[e,k] <= f_o[e,o]
  for e in dest_links | comp_in_links:
    model += R_cons[k]*f_k[e,k] <= f_o[e,o]
for e in links:
  model += lpSum([f_o[e,o] for o in objects]) == f_e[e]


# Communication Capacity Constraints
for e in links:
  model += f_e[e] <= y[e]*c_res[e] # total flow on link e less or equal than total allocated rate (number of res blocks * cap/block)
  model += y[e] <= c[e] #number of allocated res blosck less or equal than max number of res blocks

# Computation Capacity Constraints
for u in {1,2,3,6}:  # the sum of the computation flows (load of all comp links) at a given node should be less or equal than the node comp capacity
  model += lpSum([y[e] for e in (out_links[u] & comp_in_links)]) <= c_comp_in[u]
  model += lpSum([y[e] for e in (in_links[u] & comp_out_links)]) <= c_comp_out[u]


# Service Delay Constraints
d_k = LpVariable.dicts("d_k", [k for k in cmds], lowBound=0) #  variable to compute local delay per commodity
d_k_T = LpVariable.dicts("d_k_T", [k for k in cmds], lowBound=0) # variable to compute aggregate delay per commodity
# compute local delay for all cmds
for k in cmds:
  model += d_k[k] == lpSum(delay_e[e]*f_k[e,k] for e in links)
# set total delay equal to local delay for source cmds
for k in source_cmds:
  model += d_k_T[k] == d_k[k]
# compute total delay for the remaining cmds as local delay plus max over total delay of input cmds
for k in cmds - source_cmds:
  for l in in_cmds[k[0]]:
    model += d_k_T[k] >= d_k[k] + d_k_T[l]
# Constrain the total delay of the destination cmds
for k in dest_cmds:
  model += d_k_T[k] <= d_max[k] # max delay constraint per dest cmd


##### Define Objective Function
# Minimize total resource cost
# The delay can be included in the objective function or just leave it as a constraint
#model += lpSum([w[e]*y[e] for e in links]) # total cost
model += lpSum([w[e]*y[e] for e in links]) + lpSum(d_k_T[k] for k in dest_cmds) # total cost + service (det_cmd) delay



In [ ]:
##### Solve opt model

status = model.solve()
print("Feasible solution:", status)

# Objective Function Solution
print("\nTotal objective value:", model.objective.value())


# Print flow solution

# print end-to-end delay for dest cmds
for k in dest_cmds:
  #print total delay
  print("\nDestination cmd:", k, "Total delay:", d_k_T[k].varValue)

#go over all cmds
for k in sorted(cmds):
  # Print local delay, total delay, and object associated with cmd k
  o = serv.edges[k]['obj']
  print("\nCommodity:", k, "delay_k:", d_k[k].varValue, d_k_T[k].varValue, "Object:",o)
  # obtain path associated with cmd k
  # first, obtain production node for cmd k
  for e in source_links | comp_out_links:
    if f_k[e,k].varValue > 0:
      source_node = e[0]
      break
  # traverse cloud network graph from source_node
  flag = 0
  for e in nx.edge_bfs(cloudnet, source=source_node): # edge_bfs (as opposed to bfs_edges) seems to traverse all the edges, including the ones that close cycles
    if f_k[e,k].varValue > 0:
      print("Link:",e, "f_k:", f_k[e,k].varValue,"f_o:", f_o[e,o].varValue, "delay_e:", delay_e[e])
      #if e in comp_in_links:
        #print("csf function " + str(k[1]) + " =", comp_scaling[k[1]])
      #if e in comp_out_links:
       # print("nsf function " + str(k[0]) + " =", comm_scaling[k[0]])
      if e in dest_links | comp_in_links: # upon reaching the consumption link, end of path
        flag = 1
        break
  # If it does not find a consumption node, the consumption node should be the same as the prod node (self loop)
  if flag == 0: # with edge_bfs, there seems to b no need of  checking this special case
    # print incoming consumption link as the second and last hop of the path (self loop)
    for e in in_links[source_node]:
      print("Path is a self-loop")
      print("Link:",e, "f_k:", f_k[e,k].varValue,"f_o:", f_o[e,o].varValue, "delay_e:", delay_e[e])


# Print resource allocation solution
for e in sorted(links):
  if f_e[e].varValue > 0:
    print("\nLink:",e, "Flow:",f_e[e].varValue, "Resource blocks:",y[e].varValue, "Rate per block:", c_res[e], "Allocated rate:", y[e].varValue*c_res[e], "Cost:", y[e].varValue*w[e])



Feasible solution: 1

Total objective value: 734.0

Destination cmd: (7, 10) Total delay: 40.0

Destination cmd: (6, 9) Total delay: 30.0

Destination cmd: (5, 8) Total delay: 20.0

Commodity: (1, 2) delay_k: 10.0 10.0 Object: 1
Link: (40, 4) f_k: 1.0 f_o: 15.0 delay_e: 0
Link: (4, 1) f_k: 1.0 f_o: 15.0 delay_e: 10
Link: (1, 11) f_k: 1.0 f_o: 15.0 delay_e: 0

Commodity: (1, 3) delay_k: 20.0 20.0 Object: 1
Link: (40, 4) f_k: 1.0 f_o: 15.0 delay_e: 0
Link: (4, 1) f_k: 1.0 f_o: 15.0 delay_e: 10
Link: (1, 2) f_k: 1.0 f_o: 15.0 delay_e: 10
Link: (2, 21) f_k: 1.0 f_o: 15.0 delay_e: 0

Commodity: (1, 4) delay_k: 20.0 20.0 Object: 1
Link: (40, 4) f_k: 1.0 f_o: 15.0 delay_e: 0
Link: (4, 1) f_k: 1.0 f_o: 15.0 delay_e: 10
Link: (1, 2) f_k: 1.0 f_o: 15.0 delay_e: 10
Link: (2, 21) f_k: 1.0 f_o: 15.0 delay_e: 0

Commodity: (2, 5) delay_k: 0.0 10.0 Object: 2
Link: (11, 1) f_k: 1.0 f_o: 50.0 delay_e: 0
Link: (1, 12) f_k: 1.0 f_o: 50.0 delay_e: 0

Commodity: (3, 6) delay_k: 0.0 20.0 Object: 3
Link: (21

In [ ]:
# Define the element of the service graph for the cytoscape object
nodesS = [
    {
        'data': {'id': str(node), 'label': str(node)},
    }
    for node in serv.nodes
]

edgesS = [
    {
        'data': {'source': str(edge[0]), 'target': str(edge[1])},
    }
    for edge in serv.edges
]

elementsS = nodesS + edgesS


# Define the positions of the Cloud Network Graph
posG = {
    # Group 1
    1: [0.1, -0.8],
    11: [-0.7, -1.1],
    12: [-0.35, -1.85],
    13: [0.45, -1.8],

    # Group 2
    2: [0.1, 0.12],
    21: [-0.7, 0.5],
    22: [-0.3, 1.2],
    23: [0.45, 1.1],

    # Group 3
    6: [0.1, 1.80],
    61: [-0.7, 2.1],
    62: [-0.35, 2.85],
    63: [0.45, 2.8],

    # Group 4
    3: [-2.0, 0],
    30: [-1.8, -0.85],
    31: [-2.34, -0.66],
    32: [-2.65, -0.40],
    33: [-2.8, 0.03],
    34: [-2.65, 0.33],
    35: [-2.4, 0.63],
    36: [-2.0, 0.70],

    # Group 5
    4: [1.7, -0.8],
    40: [2.6, -1.1],
    49: [2.6, -0.5],

    # Group 6
    5: [1.7, 0.12],
    50: [2.6, -0.1],
    59: [2.6, 0.5],

    # Group 7
    7: [1.7, 1.80],
    70: [2.6, 1.3],
    79: [2.6, 2.3],
}


# Define the elements of the Cloud Network Graph
nodesCN = [
    {
        'data': {'id': str(node), 'label': str(node)},
        'position': {'x': posG[node][0]*200, 'y': posG[node][1]*200}
    }
    for node in cloudnet.nodes
]

edgesCN = [
    {
        'data': {'source': str(edge[0]), 'target': str(edge[1])},
    }
    for edge in cloudnet.edges
]

elementsCN = nodesCN + edgesCN



# Define a structure "Trajectory" where the keys are the cmds and the value is the path with positive flow on the cloudnet
trajectory = {}
for k in serv.edges:
  trajectory[k] = []
  for e in sorted(cloudnet.edges):
    if f_k[e,k].varValue > 0:
      trajectory[k].append(e)
from dash import Dash, html, dcc
import dash_cytoscape as cyto
from dash.dependencies import Input, Output

# Create the dash app
app = Dash(__name__)

# Define the app layout:
# Div for Cloud Network Graph
# Div for Service Graph,
# Div for dcc.RadioItems for showing the flow solution
# Div for dcc.RadioItems for showing the flow of a selected commodity
app.layout = html.Div([

  # Div for Cloud Network Graph
  html.Div([
    html.P("Cloud Network Graph:"),
    cyto.Cytoscape(
      id='cytoscape-cn',
      elements=elementsCN,
      # layout={'name': 'breadthfirst'},
      layout={'name': 'preset'},

    )
  ], style={'display': 'inline-block', 'width': '50%'}),


  # Div for Service Graph
  html.Div([
    html.P("Service Graph:"),
    cyto.Cytoscape(
      id='cytoscape-sg',
      elements=elementsS,
      layout={'name': 'breadthfirst'},
      stylesheet=[
                  {
                  'selector': 'node',
                  'style': {
                      'background-color': 'data(color)',
                      'label': 'data(label)',
                      'text-halign':'center',
                      'text-valign':'center',
                      'width':'label',
                      'height':'label',
                      }
                  },
                  {
                  'selector': 'edge',
                  'style': {
                      'curve-style': 'bezier',
                      'target-arrow-shape': 'triangle',

                      }
                  },
              # Add other styles for different types of edges if necessary

          ],
    )
  ], style={'display': 'inline-block', 'width': '50%'}),


  # Div for show the total load solution
  html.Div([
  html.Label('Show Total Load'),
  dcc.RadioItems(
    id='showtotalload',
    options=[
        {'label': 'Yes', 'value': 'yes'},
        {'label': 'No', 'value': 'no'}
    ],
    value='no',  # Default value: No
    )
], style={'position': 'absolute', 'top': '20px', 'right': '50px'}),



  # Div for show the load of a selected commodity
  html.Div([
  html.Label('Show Selected Commodity Object Flow'),
  dcc.RadioItems(
      id='show_sel_cmd_flow',
      options=[
          {'label': 'Yes', 'value': 'yes'},
          {'label': 'No', 'value': 'no'}
      ],
      value='yes',  # Default value: yes
      labelStyle={'display': 'block'}
  )
], style={'position': 'absolute', 'top': '100px', 'right': '50px'}),


  ])

# app.run_server(use_reloader=False)
@app.callback(
        Output('cytoscape-cn', 'stylesheet'),       # Edit the elment of the network graph (highlight the path)
        Input('showtotalload', 'value'),            # Get the "Total Load" value (yes or no)
        Input('cytoscape-sg', 'tapEdgeData'),       # Get the selected commodity
        Input('show_sel_cmd_flow', 'value'),        # Get the "Show Commodity Flow" value (yes or no)
    )
def show_total_load(value, tapped_cmd, show_sel_cmd):
  # Create an updated stylesheet that will be the output
  updated_stylesheet=[
      # Group selectors Node
      {
          'selector': 'node',
          'style': {
              'content': 'data(label)',
              'color':'white',
              'font-size':'30px',
          }
      },

      # Group selectors Edges
      {
          'selector': 'edge',
          'style': {
              'curve-style': 'bezier',
              'target-arrow-shape': 'triangle',
          }
      },
  ]

  # If the value of the radio-item is yes,
  # check for every link if it has positive flow
  # If so, we define the style for the link which will be blue in color and will have total load (f_e) as its label.
  if value == 'yes':
    for e in cloudnet.edges:
      if f_e[e].varValue>0:
        edge_style = {
          'line-color': 'blue',
          'target-arrow-color': 'blue',
          'label': f"{f_e[e].varValue}",
          'color':'white',
          'font-size':'20px',
        }
        # select the link that has positive flow (source=e[0]. target=e[1])
        # and change its edge style
        updated_stylesheet.append({
            'selector': f'edge[source="{e[0]}"][target="{e[1]}"]',
            'style': edge_style
        })


  # If "Show selected Commodity Flow" is "yes" and a cmd has been selected
  if tapped_cmd is not None and show_sel_cmd == "yes":
    #Obtain source and target function of the selected cmd and obtain their ids
    source_node = tapped_cmd['source']
    target_node = tapped_cmd['target']
    src_id=int(source_node)
    targ_id=int(target_node)
    commodity=(src_id,targ_id)

    # Obtain the trajectory of the commodity k in the network graph
    path = trajectory[commodity]
    for e in path:
      curr_f_o = f_o[e,serv.edges[commodity]["obj"]].varValue
      edge_style = {
        'line-color': 'red',
        'target-arrow-color': 'red',
        'target-arrow-shape': 'triangle',
        'label': f"{curr_f_o}",
        # 'arrowhead': 'vee'
    }
      updated_stylesheet.append({
              'selector': f'edge[source="{str(e[0])}"][target="{str(e[1])}"]',
              'style': edge_style
          })

  return updated_stylesheet

In [ ]:
app.run_server(debug=True, port=8052)

<IPython.core.display.Javascript object>